In [1]:
%load_ext autoreload
%autoreload 2

import pyscan as ps
from pyscan.drivers.testing.auto_test_driver import test_driver
from pyvisa import ResourceManager, VisaIOError
import pytest

Could not load Keysight SD1
Could not load Keysight SD1
pylablib not found, AttocubeANC350 not loaded
Basler Camera software not found, BaserCamera not loaded
Helios Camera not installed
msl not installed, Thorlabs BSC203 driver not loaded
seabreeze module not found, Ocean Optics not imported
Thorlabs Kinesis not found, ThorlabsBSC203 not loaded
Thorlabs Kinesis not found, ThorlabsBPC303 not loaded
Thorlabs Kinesis not found, ThorlabsMFF101 not loaded


In [2]:
rm = ResourceManager()

rs = rm.list_resources()
# print(rs)
for r in rs:
    res = rm.open_resource(r)
    try:      
        name = res.query('*IDN?')
        if 'SR830' in name:
            print(r, name)
            break
    except VisaIOError:
        pass
    res.close()

GPIB0::8::INSTR Stanford_Research_Systems,SR830,s/n86813,ver1.07 



In [3]:
name

'Stanford_Research_Systems,SR830,s/n86813,ver1.07 \n'

In [4]:
srs830 = ps.Stanford830(res)

test_driver(srs830)

Initial state for the Stanford830 was: [('id', 'Stanford_Research_Systems,SR830,s/n86813,ver1.07 '), ('phase', 0.0), ('reference_source', 'external'), ('frequency', 1000.0), ('reference_slope', 'sine zero'), ('harmonic', 1), ('amplitude', 0.496), ('input_configuration', 'A-B'), ('input_ground', 'AC'), ('input_coupling', 'AC'), ('input_line_filter', 'none'), ('sensitivity', 2e-09), ('reserve_mode', 'high'), ('time_constant', 30), ('filter_slope', 6), ('synchronous_filter', 'off'), ('display1_output_source', 'x'), ('display2_output_source', 'y'), ('auxillary_voltage1', -10.5), ('auxillary_voltage2', -10.5), ('auxillary_voltage3', -10.5), ('auxillary_voltage4', -10.5), ('sample_rate', 0.0625), ('end_buffer_mode', 'one shot'), ('trigger_mode', 'off')]
These blacklisted settings and their corresponding values were not reset:  [('_amplitude', 0.496), ('_input_configuration', 'A-B'), ('_time_constant', 30)]
Reset state for the Stanford830 was: [('id', 'Stanford_Research_Systems,SR830,s/n86813

In [5]:
# Test get and set display

srs830.set_display(1, 'r', 'aux1') # number, source, ratio
source, ratio = srs830.get_display(1)
assert source == 'r'
assert ratio == 'aux1'


srs830.set_display(1, 'x', 'none')
source, ratio = srs830.get_display(1)
assert source == 'x'
assert ratio == 'none'


with pytest.raises(AssertionError):
    srs830.set_display(1, 'y', 'none')
print('Sucessfully failed setting channel 1 to y')
    
    
srs830.set_display(2, 'theta', 'aux3')
source, ratio = srs830.get_display(2)
assert source == 'theta'
assert ratio == 'aux3'


srs830.set_display(2, 'y', 'none')
source, ratio = srs830.get_display(2)
assert source == 'y'
assert ratio == 'none'


with pytest.raises(AssertionError):
    srs830.set_display(2, 'x', 'none')
print('Sucessfully failed setting channel 2 to x')

print("Successfully tested get and set display")


Sucessfully failed setting channel 1 to y
Sucessfully failed setting channel 2 to x
Successfully tested get and set display


In [6]:
# Test get and set offset and expand

srs830.set_channel_offset_expand(1, 100, 10)
print(srs830.get_channel_offset_expand(1))

srs830.set_channel_offset_expand(1, 0, 1)
print(srs830.get_channel_offset_expand(1))


['0', '1']
['0', '0']


In [7]:
# Test auto offset
# v0 should be some value, v_offset should be close to 0
srs830.set_channel_offset_expand(1, 0, 1)
v0 = srs830.read('x')
srs830.auto_offset('x')
v_offset = srs830.read('x')
print(v0, v_offset)
assert abs(v_offset - 0) < 0.0001


srs830.set_channel_offset_expand(2, 0, 1)
v0 = srs830.read('y')
srs830.auto_offset('y')
v_offset = srs830.read('y')
print(v0, v_offset)
assert abs(v_offset - 0) < 0.0001

srs830.set_channel_offset_expand(1, 0, 1)
v0 = srs830.read('r')
srs830.auto_offset('r')
v_offset = srs830.read('r')
print(v0, v_offset)
assert abs(v_offset - 0) < 0.0001


srs830.set_channel_offset_expand(1, 0, 1)
srs830.set_display(1, 'x', 'none')


2.18428e-09 2.18428e-09
-2.18439e-09 -2.18439e-09
2.18428e-09 2.18428e-09


In [8]:
# Test read aux inputs
# shoudl show some random small value
for i in range(1, 5):
    aux_input = srs830.read_aux_input(i)
    print(aux_input)
    assert abs(aux_input - 0) < 0.1

0.00466667
-0.001
0.00166667
0.00233333


In [9]:
# Read x, y, r, theta

print(srs830.read('x'))
print(srs830.read('y'))
print(srs830.read('r'))
print(srs830.read('theta'))

2.18428e-09
-2.18439e-09
2.18428e-09
-2597.219879105803


In [10]:
# Read display

for i in range(1, 3):
    print(srs830.read_display(i))

2.18439e-09
-2.18451e-09


In [11]:
# Test snap

for source in ['y', 'r', 'theta', 'aux1', 'aux2', 'aux3',
                  'aux4', 'frequency', 'display1', 'display2']:
    print(srs830.snap('x', source))

srs830.snap('x', 'y', 'r', 'theta', 'frequency', 'display1')


with pytest.raises(AssertionError):
    srs830.snap('x', 'y', 'r', 'theta', 'frequency', 'display1', 'display2')
    # snap only accepts 6 entries

[2.18428e-09, -2.18439e-09]
[2.18428e-09, 2.18428e-09]
[2.18428e-09, -2670.7877221406993]
[2.18428e-09, -0.003]
[2.18428e-09, -0.00566667]
[2.18428e-09, -0.008]
[2.18428e-09, -0.00833333]
[2.18428e-09, 1000.0]
[2.18428e-09, 2.18439e-09]
[2.18428e-09, -2.18451e-09]


In [12]:
# test get ID

id = srs830.get_identificaiton_string()
assert id == 'Stanford_Research_Systems,SR830,s/n86813,ver1.07 '
print(id)

Stanford_Research_Systems,SR830,s/n86813,ver1.07 


In [13]:
# test buffer_points

srs830.buffer_points
srs830.wait_for_trigger()
srs830.set_buffer_mode(0.25)
srs830.wait_for_trigger()
# srs830.start()
srs830.trigger()
sleep(20)
srs830.buffer_points


AttributeError: 'Stanford830' object has no attribute 'buffer_points'

In [ ]:
srs830.query('SRAT?')

'0\n'

In [ ]:
srs830.sample_rate

0.0625